# PyVista VTK 表面渲染

使用 PyVista 动态渲染 ASCII Legacy PolyData 文件

In [2]:
import vtk
import pyvista as pv
import numpy as np

In [3]:
# 读取 VTK 文件
reader = vtk.vtkPolyDataReader()
reader.SetFileName("out.vtk")
reader.Update()
mesh = reader.GetOutput()

print(f"点数: {mesh.GetNumberOfPoints():,}")
print(f"多边形数: {mesh.GetNumberOfPolys():,}")

点数: 76,702
多边形数: 151,298


In [4]:
# 修复和优化网格
triangle_filter = vtk.vtkTriangleFilter()
triangle_filter.SetInputData(mesh)
triangle_filter.Update()
triangle_mesh = triangle_filter.GetOutput()

# 计算法线
normals = vtk.vtkPolyDataNormals()
normals.SetInputData(triangle_mesh)
normals.ConsistencyOn()
normals.SplittingOff()
normals.AutoOrientNormalsOn()
normals.Update()
mesh_with_normals = normals.GetOutput()

print(f"修复后点数: {mesh_with_normals.GetNumberOfPoints():,}")
print(f"修复后三角形数: {mesh_with_normals.GetNumberOfPolys():,}")

修复后点数: 76,702
修复后三角形数: 151,298


In [5]:
# 转换为 PyVista 格式
pv_mesh = pv.wrap(mesh_with_normals)

print("PyVista 网格信息:")
print(pv_mesh)

# 显示包围盒信息
bounds = pv_mesh.bounds
print(f"\n包围盒:")
print(f"X: {bounds[0]:.2f} ~ {bounds[1]:.2f}")
print(f"Y: {bounds[2]:.2f} ~ {bounds[3]:.2f}")
print(f"Z: {bounds[4]:.2f} ~ {bounds[5]:.2f}")

PyVista 网格信息:
PolyData (0x164a1a020)
  N Cells:    151298
  N Points:   76702
  N Strips:   0
  X Bounds:   1.153e+01, 1.191e+02
  Y Bounds:   4.620e+01, 1.150e+02
  Z Bounds:   0.000e+00, 1.270e+02
  N Arrays:   1

包围盒:
X: 11.53 ~ 119.13
Y: 46.20 ~ 115.04
Z: 0.00 ~ 127.00


## 基础表面渲染

In [7]:
# 基础表面渲染
pv_mesh.plot(
    style='surface',
    show_edges=True,
    color='lightblue',
    lighting=True,
    window_size=[800, 600]
)

Widget(value='<iframe src="http://localhost:64864/index.html?ui=P_0x302838b50_0&reconnect=auto" class="pyvista…

## 带颜色映射的渲染

In [6]:
# 添加高度标量数据
height_array = vtk.vtkFloatArray()
height_array.SetName("Height")
height_array.SetNumberOfComponents(1)

points = mesh_with_normals.GetPoints()
for i in range(points.GetNumberOfPoints()):
    point = points.GetPoint(i)
    height_array.InsertNextValue(point[2])  # Z 坐标作为高度

mesh_with_normals.GetPointData().SetScalars(height_array)

# 重新转换为 PyVista 格式
pv_mesh_colored = pv.wrap(mesh_with_normals)

# 带颜色映射的渲染
pv_mesh_colored.plot(
    scalars="Height",
    cmap='viridis',
    style='surface',
    show_edges=False,
    lighting=True,
    window_size=[800, 600]
)

Widget(value='<iframe src="http://localhost:65085/index.html?ui=P_0x164a324d0_0&reconnect=auto" class="pyvista…

## 高级渲染选项

In [7]:
# 高级渲染：自定义光照和材质
plotter = pv.Plotter(window_size=[1000, 800])

# 添加网格
plotter.add_mesh(
    pv_mesh_colored,
    scalars="Height",
    cmap='plasma',
    style='surface',
    show_edges=False,
    lighting=True,
    metallic=0.1,
    roughness=0.3
)

# 设置光照
plotter.enable_shadows()
plotter.add_light(pv.Light(position=(100, 100, 100), intensity=0.8))
plotter.add_light(pv.Light(position=(-100, -100, 100), intensity=0.4))

# 设置相机视角
#plotter.camera_position = 'isometric'
#plotter.background = 'white'

# 显示
plotter.show()

Widget(value='<iframe src="http://localhost:65085/index.html?ui=P_0x16c1906d0_1&reconnect=auto" class="pyvista…

## 数据统计

In [11]:
# 数据统计报告
print("=== VTK 文件统计 ===")
print(f"总点数: {pv_mesh.n_points:,}")
print(f"总三角形数: {pv_mesh.n_cells:,}")
print(f"数据数组数量: {len(pv_mesh.array_names)}")
print(f"数据数组: {pv_mesh.array_names}")

# 几何信息
bounds = pv_mesh.bounds
print(f"\n=== 几何尺寸 ===")
print(f"X 尺寸: {bounds[1] - bounds[0]:.2f}")
print(f"Y 尺寸: {bounds[3] - bounds[2]:.2f}")
print(f"Z 尺寸: {bounds[5] - bounds[4]:.2f}")

# 估算表面积
area = pv_mesh.area
print(f"\n表面积: {area:.2f}")

# 体积估算（如果是封闭网格）
if pv_mesh.is_manifold:
    volume = pv_mesh.volume
    print(f"体积: {volume:.2f}")
else:
    print("网格不是封闭的，无法计算体积")

print(f"\n✅ PyVista 渲染完成！您的VTK文件包含完整的表面信息。")

=== VTK 文件统计 ===
总点数: 56,514
总三角形数: 121,512
数据数组数量: 1
数据数组: ['Normals']

=== 几何尺寸 ===
X 尺寸: 107.69
Y 尺寸: 68.85
Z 尺寸: 127.00

表面积: 40148.90
网格不是封闭的，无法计算体积

✅ PyVista 渲染完成！您的VTK文件包含完整的表面信息。
